# **Spazzata in frequenza**

Il notebook mostra come fare una scansione in frequenza per la misura del diagramma di Bode. Lo script si basa 
non su una FFT ma semmai su un fit con un coseno.

Parametri chiave

* `nper` imposta il numero di periodi misurati nell'acquisizione... dato il metodo di stima (fit), ne basta uno
* `nf` imposta il numero di frequenze nello scan
* `npt` imposta il numero dei punti da acquisire
* `f0` imposta la frequenza iniziale
* `f1` imposta la frequenza finale
* `flag_return` => attiva andata e ritorno
    
**Algoritmo di campionamento**. Cerca di acquisire `nper` periodi con `npt` punti. Per fare questo con una oscillazione a frequenza `ff` in teoria serve una frequenza di campionamento pari a

        ff * npt / nper

Tuttavia, $\texttt{Analog Discovery 2}$ può solo campionare a sottomultipli $(100/n)\,{\rm MHz}$. Lo script quindi calcola $n$ con la formula di sopra e lo arrotonda *verso l'alto*, calcolando poi quale sia l'esatto numero di punto per coprire il numero di periodi richiesto. L'arrotondamento verso l'alto è stato scelto in modo che il numero di punti finale non superi mai quello impostato nei parametri.

**Attenzione**. La qualità dello studio declina rapidamente sopra i $100\,{\rm kHz}$ ed è certamente sconsigliato andare sopra $1\,{\rm MHz}$. Il motivo più probabile per le discrepanze che si sviluppano è legato al cross-talk fra i cavi del *bundle* fornito, che non è particolarmente adatto alle alte frequenze. 

In [8]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
import time
    
# -[Inizializzazione parametri]------------------------------------------------------

#   1. Flag di Controllo
flag_return = True
flag_show = True
flag_save = True

#   2. Parametri di sampling
filename = f"IV_{time.strftime('%Y%m%d_%H%M%S')}"
nper = 4
nv = 10
npt = 8192
ov = np.linspace(-5, 5, nv)


#   3. Misure e valori di riferimento
R = 1e4
expected_Vj = 0.7


# -[Configurazione AD2]--------------------------------------------------------

#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()

#   2. Configurazione generatore di funzioni
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.func = tdwf.funcDC
wavegen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 1e6
scope.npt = npt
scope.ch1.rng = 10
scope.ch2.rng = 10



#-[Ciclo di misura]------------------------------------------------------------
#   1. Creazione figura e link agli eventi
fig, [[ax1, ax3], [ax2, ax4a]] = plt.subplots(2, 2,figsize=(12,6),                                
    gridspec_kw={'width_ratios': [1, 2]})
ax4b = ax4a.twinx()
fig.canvas.manager.set_window_title('Spazzata tensione')


#   2. Ciclo di misura
flag_first = True
Vd = np.full((nv, 2), np.nan)
Vr = np.full((nv, 2), np.nan)
Id = np.full((nv, 2), np.nan)
for ar in range(2 if flag_return else 1):  # go and return loop
    for ii in range(len(ov)):  # frequency loop
        
        # 1. Impostazione dlela frequenza e del sampling

        if ar==0:
            findex = ii
        else:
            findex = len(ov)-ii-1
        oo = ov[findex]
#        scope.trig(True, hist = 0.1)

        wavegen.w1.offs = ov[findex]  # Vpp
        wavegen.w1.start()
        time.sleep(0.1) # wait for the signal to stabilize

        # 2. Campionamento e analisi risultati

        scope.sample()
        fitfuncDC = lambda x,o: np.array( [o for i in x] )
        pp1,cm1 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch1.vals, p0=[wavegen.w1.offs])
        pp2,cm2 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch2.vals, p0=[wavegen.w1.offs])
        #epp1 = np.sqrt(np.diagonal(cm1))
        #epp2 = np.sqrt(np.diagonal(cm2))

        # 3. Aggiornamento dei dati

        Vd[findex, ar] = pp1[0]
        Vr[findex, ar] = pp2[0]
        Id[findex, ar] = pp2[0] / R

        # 4. Aggiornamento plots

        if flag_first:
            flag_first = False
            if flag_show:
                hp1_true, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                hp1_sym, = ax1.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp1), "--", label="Fit Ch1", color="#00e7e7")
                ax1.grid(linestyle='-.')
                ax1.set_ylabel("Vd [V]", fontsize=15)
                ax1.set_ylim([-1.2*max(ov), 1.2*max(ov)])
                ax1.set_title(f"Time plots", fontsize=15)


                hp2_true, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                hp2_sym, = ax2.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp2), "--", label="Fit Ch2", color="#e500a7")
                ax2.grid(linestyle='-.')
                ax2.set_xlabel("Time [msec]", fontsize=15)
                ax2.set_ylabel("Vr [A]", fontsize=15)
                ax2.set_ylim([-1.2*max(ov), 1.2*max(ov)])


                hp3A, = ax3.plot(Vd[:, 0], Id[:, 0], ".", markerfacecolor = "none", label="Go Run", color="tab:purple")
                if flag_return:
                    hp3R, = ax3.plot(Vd[:, 1], Id[:, 1], "v",  markerfacecolor = "none", label="Return Run", color="tab:cyan")
                
                ax3.grid(linestyle='-.')
                ax3.set_xlabel("Vd [V]", fontsize=15)
                ax3.set_ylabel("Id = Vr/R [A]", fontsize=15)
                ax3.set_xlim([min(ov)-0.5, expected_Vj])
                ax3.set_ylim([min(ov)/R-0.2/R, max(ov)/R])
                ax3.legend(loc='upper left')
                ax3.set_title(f"I-V Charateristic", fontsize=15)


                hp4VA, = ax4a.plot(ov, Vd[:, 0], ".", markerfacecolor = "none", label="Vd Go", color="tab:orange")
                if flag_return:
                    hp4VR, = ax4a.plot(ov, Vd[:, 1], "v",  markerfacecolor = "none", label="Vd Return", color="tab:orange")                
                ax4a.grid(linestyle='-.')
                ax4a.set_xlabel("Vcc [V]", fontsize=15)
                ax4a.set_ylabel("Vd [V]", fontsize=15)
                ax4a.tick_params(axis='y', labelcolor="tab:orange")
                ax4a.set_xlim([min(ov)-0.5, 1.2*max(ov)])
                ax4a.set_ylim([min(ov)-0.5, 1.2*expected_Vj])

                
                hp4IA, = ax4b.plot(ov, Vr[:, 0], ".", markerfacecolor = "none", label="Id Go", color="tab:blue")
                if flag_return:
                    hp4IR, = ax4b.plot(ov, Vr[:, 1], "v",  markerfacecolor = "none", label="Id Return", color="tab:blue")
                ax4b.set_ylabel("Id = Vr/R [A]", fontsize=15)
                ax4b.tick_params(axis='y', labelcolor="tab:blue")
                ax4b.set_ylim([min(ov)/R-0.2/R, max(ov)/R])



                plt.tight_layout()
                plt.show(block=False)    
        else:
            if flag_show:
                hp1_true.set_xdata(1000*scope.time.vals)
                hp1_true.set_ydata(scope.ch1.vals)
                hp1_sym.set_xdata(1000*scope.time.vals)
                hp1_sym.set_ydata(fitfuncDC(scope.time.vals, *pp1))
                hp2_true.set_xdata(1000*scope.time.vals)
                hp2_true.set_ydata(scope.ch2.vals)
                hp2_sym.set_xdata(1000*scope.time.vals)
                hp2_sym.set_ydata(fitfuncDC(scope.time.vals, *pp2))
                if ar==0:
                    hp3A.set_xdata(Vd[:, 0])
                    hp3A.set_ydata(Id[:, 0])

                    hp4VA.set_xdata(ov[:])
                    hp4VA.set_ydata(Vd[:, 0])
                    hp4IA.set_xdata(ov[:])
                    hp4IA.set_ydata(Id[:, 0])
                else:
                    hp3R.set_xdata(Vd[:, 1])
                    hp3R.set_ydata(Id[:, 1])

                    hp4VR.set_xdata(ov[:])
                    hp4VR.set_ydata(Vd[:, 1])
                    hp4IR.set_xdata(ov[:])
                    hp4IR.set_ydata(Id[:, 1])



                fig.canvas.draw()
                fig.canvas.flush_events()

# ---------------------------------------
ad2.close()

# - [Data saving]--------------------------------------------------------------
if flag_save:
    info = f"IV curve: {filename}\n"
    if flag_return:
        data_to_save = np.c_[ov, Vd, Id]
        info += f"Vcc[V]\tVd_Go[V]\tVd_Return[V]\tId_Go[A]\tId_Return[A]\n"
    else:
        data_to_save = np.c_[ov, Vd[:, 0], Id[:, 0]]
        info += "Vcc[V]\tVd[V]\tId[A]\n\n"
    np.savetxt("../../../Data/"+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")
    print("Data saved")



Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.
Data saved


In [22]:
ad2.close()

Dispositivo disconnesso.
